In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# ===============================
# STEP 1: Setup
# ===============================
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, models

# Use GPU if available
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", device)

# ===============================
# STEP 2: Dataset & Dataloaders
# ===============================
data_dir = "/content/drive/MyDrive/dataset"

# Training augmentations
train_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225]),
    transforms.RandomResizedCrop(224, scale=(0.8, 1.0))
])

# Validation = no heavy augmentation
val_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

train_dataset = datasets.ImageFolder(root=f"{data_dir}/train", transform=train_transform)
val_dataset   = datasets.ImageFolder(root=f"{data_dir}/val",   transform=val_transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=2)
val_loader   = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=2)

print("Classes:", train_dataset.classes)

# ===============================
# STEP 3: Model (MobileNetV2)
# ===============================
model = models.mobilenet_v2(weights="IMAGENET1K_V1")

# Freeze feature extractor
for param in model.features.parameters():
    param.requires_grad = False

# Replace classifier head with new one (your classes)
num_classes = len(train_dataset.classes)
model.classifier[1] = nn.Linear(model.last_channel, num_classes)

model.load_state_dict(torch.load("/content/drive/MyDrive/insect_model.pth", map_location=device))
model = model.to(device)
model.train()

# ===============================
# STEP 4: Training setup
# ===============================
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
epochs = 10  # start small, adjust later

# ===============================
# STEP 5: Training loop
# ===============================
for epoch in range(epochs):
    # --- Train ---
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    # --- Validate ---
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

    val_acc = 100 * correct / total
    print(f"Epoch {epoch+1}/{epochs}, "
          f"Loss: {running_loss/len(train_loader):.3f}, "
          f"Val Acc: {val_acc:.2f}%")

# ===============================
# STEP 6: Save model
# ===============================
torch.save(model.state_dict(), "/content/drive/MyDrive/insect_model.pth")
print("Model saved to Drive as insect_model.pth ✅")


Using device: cuda
Classes: ['beetle', 'butterfly', 'spider', 'wasps and bees']
Epoch 1/10, Loss: 0.836, Val Acc: 58.20%
Epoch 2/10, Loss: 0.839, Val Acc: 60.66%
Epoch 3/10, Loss: 0.853, Val Acc: 61.89%
Epoch 4/10, Loss: 0.812, Val Acc: 62.70%
Epoch 5/10, Loss: 0.813, Val Acc: 61.48%
Epoch 6/10, Loss: 0.801, Val Acc: 61.07%
Epoch 7/10, Loss: 0.798, Val Acc: 58.61%
Epoch 8/10, Loss: 0.814, Val Acc: 61.48%
Epoch 9/10, Loss: 0.797, Val Acc: 62.70%
Epoch 10/10, Loss: 0.777, Val Acc: 61.89%
Model saved to Drive as insect_model.pth ✅


In [ ]:
import torch
from torchvision import transforms, models
from PIL import Image
import matplotlib.pyplot as plt



# ===============================
# STEP 1: Load the trained model
# ===============================
device = "cuda" if torch.cuda.is_available() else "cpu"

# Classes (must match your dataset order)
class_names = ["beetle", "butterfly", "spider", "wasps and bees"]

# Rebuild MobileNetV2
model = models.mobilenet_v2(weights=None)
num_classes = len(class_names)
model.classifier[1] = torch.nn.Linear(model.last_channel, num_classes)

# Load trained weights
model.load_state_dict(torch.load("/content/drive/MyDrive/insect_model.pth", map_location=device))
model = model.to(device)
model.eval()

# ===============================
# STEP 2: Transform for test images
# ===============================
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

# ===============================
# STEP 3: Prediction function with visualization
# ===============================
def predict_image(image_path):
    # Load image
    img = Image.open(image_path).convert("RGB")
    img_tensor = transform(img).unsqueeze(0).to(device)

    # Model prediction
    with torch.no_grad():
        outputs = model(img_tensor)
        _, predicted = outputs.max(1)

    predicted_class = class_names[predicted.item()]

    # Show image + prediction
    plt.imshow(img)
    plt.axis("off")
    plt.title(f"Prediction: {predicted_class}", fontsize=14, color="green")
    plt.show()

    return predicted_class

# ===============================
# STEP 4: Upload and test
# ===============================
from google.colab import files
uploaded = files.upload()  # choose an image (jpg/png)

# Predict first uploaded file
test_img_path = list(uploaded.keys())[0]
predict_image(test_img_path)


KeyboardInterrupt: 